In [2]:
import pandas as pd

In [6]:
DF = pd.read_csv('C:/Users/username/Dir/IotPB/Data/Raw/room_temp_rawdata.csv')
display(DF)

,Timestamp,Temperature_Celsius,Relative_Humidity
0,2021-01-09 12:22:54,13.4,55
1,2021-01-09 12:24:58,15.8,46
2,2021-01-09 12:26:58,17.3,45
3,2021-01-09 12:29:01,19.0,42
4,2021-01-09 12:31:16,20.1,39
...,...,...,...
7011,2021-05-14 11:51:53,25.5,52
7012,2021-05-14 12:21:58,25.4,52
7013,2021-05-14 12:52:00,25.4,52
7014,2021-05-14 13:22:06,25.6,51


Nome da coluna | Descrição
-------------- | ----------
Timestamp| Coluna que representa a data da amostra.
Temperature_Celsius| Coluna que representa a temperatura em Celsius do quarto.
Relative_Humidity| Coluna que representa a umidade do quarto.

>Segundo a NR17, do Ministério do Trabalho, a temperatura do ambiente de trabalho onde são executadas atividades intelectuais como nos laboratórios, escritórios, sala de desenvolvimento e projetos deve ficar entre 20 e 23 graus centígrados, com umidade relativa inferior a 40%. Outra norma, a ISO 9241, estabelece que o ideal é manter a temperatura entre 20 e 24 graus no verão e 23 e 26 no inverno, com umidade relativa entre 40% e 80%.

#### Funções

In [7]:
from datetime import datetime

def pass_time_to_season(date_string, hemisphere):
    datestring = datetime.fromisoformat(date_string)
    seasons = ['primavera','verao','outono','inverno']
    md = datestring.month * 100 + datestring.day
    if ((md > 320) and (md < 621)):
        s = 0 #spring
    elif ((md > 620) and (md < 923)):
        s = 1 #summer
    elif ((md > 922) and (md < 1223)):
        s = 2 #fall
    else:
        s = 3 #winter
    if hemisphere != 'north':
        if s < 2:
            s += 2 
        else:
            s -= 2

    return seasons[s]

# print(pass_time_to_season('2021-01-09 12:22:54','south'))

def is_temp_ideal(temperature, season):
  if season == 'verao' or season == 'outono':
    ideal = [20,21,22,23]
  else:
    ideal = [23,24,25,26]
  
  if temperature in ideal:
    return True
  else:
    return False
# is_temp_ideal(25.4,'verao')

def temp_target(temperature, season):
  if season == 'verao' or season == 'outono':
    ideais = [20,23]
    target = ideais[0] if ideais[0] > temperature else ideais[1]
  else:
    ideais = [23,26]
    target = ideais[0] if ideais[0] > temperature else ideais[1]
  return target

#### Adicionando colunas necessarias para operação

In [8]:
df_with_season = DF.copy()
df_with_season['Season'] = df_with_season['Timestamp'].apply(pass_time_to_season, hemisphere='south')
display(df_with_season)

,Timestamp,Temperature_Celsius,Relative_Humidity,Season
0,2021-01-09 12:22:54,13.4,55,verao
1,2021-01-09 12:24:58,15.8,46,verao
2,2021-01-09 12:26:58,17.3,45,verao
3,2021-01-09 12:29:01,19.0,42,verao
4,2021-01-09 12:31:16,20.1,39,verao
...,...,...,...,...
7011,2021-05-14 11:51:53,25.5,52,outono
7012,2021-05-14 12:21:58,25.4,52,outono
7013,2021-05-14 12:52:00,25.4,52,outono
7014,2021-05-14 13:22:06,25.6,51,outono


In [9]:
df_with_ideal_classification = df_with_season.copy()
df_with_ideal_classification['is_ideal_temperature'] = df_with_ideal_classification.apply(lambda x: is_temp_ideal(x.Temperature_Celsius, x.Season),axis=1)
display(df_with_ideal_classification.head(3))

,Timestamp,Temperature_Celsius,Relative_Humidity,Season,is_ideal_temperature
0,2021-01-09 12:22:54,13.4,55,verao,False
1,2021-01-09 12:24:58,15.8,46,verao,False
2,2021-01-09 12:26:58,17.3,45,verao,False


In [10]:
df_with_target_temperature = df_with_ideal_classification.copy()
df_with_target_temperature['target_temp'] = df_with_target_temperature[df_with_target_temperature['is_ideal_temperature'] == False] \
.apply(lambda x: temp_target(x.Temperature_Celsius, x.Season),axis=1)
display(df_with_target_temperature)

,Timestamp,Temperature_Celsius,Relative_Humidity,Season,is_ideal_temperature,target_temp
0,2021-01-09 12:22:54,13.4,55,verao,False,20.0
1,2021-01-09 12:24:58,15.8,46,verao,False,20.0
2,2021-01-09 12:26:58,17.3,45,verao,False,20.0
3,2021-01-09 12:29:01,19.0,42,verao,False,20.0
4,2021-01-09 12:31:16,20.1,39,verao,False,23.0
...,...,...,...,...,...,...
7011,2021-05-14 11:51:53,25.5,52,outono,False,23.0
7012,2021-05-14 12:21:58,25.4,52,outono,False,23.0
7013,2021-05-14 12:52:00,25.4,52,outono,False,23.0
7014,2021-05-14 13:22:06,25.6,51,outono,False,23.0


#### Salvando novo dataframe em um .csv

Nome da coluna | Descrição
-------------- | ----------
Timestamp| Coluna que representa a data da amostra.
Temperature_Celsius| Coluna que representa a temperatura em Celsius do quarto.
Relative_Humidity| Coluna que representa a umidade do quarto.
Season| Coluna que representa a estão do ano que a leitura se encontra.
is_ideal_temperature| Coluna boleana para facilitar filtros de analise indicando se a leitura está na temperatura ideal.
target_temp| Coluna que representa o valor desejado de temperatura para a leitura.

In [12]:
dirstring = 'C:/Users/username/Dir/IotPB/Data/Processed/'
df_with_target_temperature.to_csv(f"{dirstring}room_temp_prepdata.csv", encoding='utf-8')
print("Arquivo salvo com sucesso!")

Arquivo salvo com sucesso!
